grasp

In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)

In [2]:
gs.init(backend=gs.cpu, seed=42)

[Genesis] [18:22:22] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [18:22:22] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [18:22:22] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [18:22:24] [INFO] Running on [13th Gen Intel(R) Core(TM) i7-13800H] with backend gs.cpu. Device memory: 31.59 GB.
[Genesis] [18:22:24] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: 42, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.


In [3]:

scene = gs.Scene(
    show_viewer = True,
    viewer_options = gs.options.ViewerOptions(
        res           = (1280, 960),
        camera_pos    = (3.5, 0.0, 2.5),
        camera_lookat = (0.0, 0.0, 0.5),
        camera_fov    = 40,
        max_FPS       = 60,
    ),
    sim_options=gs.options.SimOptions(
        dt = 0.01,
    ),
    vis_options = gs.options.VisOptions(
        show_world_frame = True,
        world_frame_size = 1.0,
        show_link_frame  = False,
        show_cameras     = False,
        plane_reflection = True,
        ambient_light    = (0.1, 0.1, 0.1),
    ),
    renderer=gs.renderers.Rasterizer(),
)

plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka = scene.add_entity(
    gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
)
cube_size = (0.1, 0.07, 0.03)
cube_pos = (0.7, 0, 0.015)

cube = scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = cube_pos,
    ),
    # surface=gs.textures.ColorTexture(
    #     color = (0.8, 0.1 , 0.1),
    # ),
)
cube.surface.color = (0.8, 0.1 , 0.1)

cylinder_pos = (0.7, -0.2, 0.05)
cylinder_radius = 0.03
cylinder_height = 0.1

cylinder = scene.add_entity(
    gs.morphs.Cylinder(
        radius = cylinder_radius,
        height = cylinder_height,
        pos    = cylinder_pos,
    ),
    # surface=gs.textures.ColorTexture(
    #     color = (0.2, 0.8, 0.2),        
    # ),
)
cylinder.surface.color = (0.2, 0.8, 0.2)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (3.5, 0.0, 2.5),
    lookat = (0, 0, 0.5),
    fov    = 30,
    GUI    = False,
)


[Genesis] [18:22:28] [INFO] Scene <aa74244> created.
[Genesis] [18:22:28] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <36c99a3>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [18:22:28] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <4611a6b>, morph: <gs.morphs.MJCF(file='d:\envs\pygen\Lib\site-packages\genesis\assets\xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [18:22:29] [WARNING] (MJCF) Tendon not supported
[Genesis] [18:22:30] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [18:22:30] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [18:22:30] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [18:22:30] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.

In [4]:

scene.build()
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)

# set control gains
# Note: the following values are tuned for achieving best behavior with Franka
# Typically, each new robot would have a different set of parameters.
# Sometimes high-quality URDF or XML file would also provide this and will be parsed.
franka.set_dofs_kp(
    np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
)
franka.set_dofs_kv(
    np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
)
franka.set_dofs_force_range(
    np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
    np.array([ 87,  87,  87,  87,  12,  12,  12,  100,  100]),
)

# render rgb, depth, segmentation, and normal
# rgb, depth, segmentation, normal = cam.render(rgb=True, depth=True, segmentation=True, normal=True)


[Genesis] [18:22:33] [INFO] Building scene <aa74244>...
[Genesis] [18:22:37] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [18:22:37] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.
[Genesis] [18:22:57] [INFO] Compiling simulation kernels...
[Genesis] [18:28:06] [INFO] Building visualizer...
[Genesis] [18:28:17] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [5]:
# get cubes prperties
cube_pos = cube.get_pos()
print(f"cube_pos: {cube_pos}")
cube_rot = cube.get_quat()
print(f"cube_rot: {cube_rot}")
cube_mass = cube.get_mass()
print(f"cube_mass: {cube_mass}")
cube_vertices = cube.get_verts()
print(f"cube_vertices: {cube_vertices}")

cube_pos: tensor([0.7000, 0.0000, 0.0150])
cube_rot: tensor([1., 0., 0., 0.])
cube_mass: 0.04200000000000001
cube_vertices: tensor([[ 0.6500, -0.0350,  0.0000],
        [ 0.6500, -0.0350,  0.0300],
        [ 0.6500,  0.0350,  0.0000],
        [ 0.6500,  0.0350,  0.0300],
        [ 0.7500, -0.0350,  0.0000],
        [ 0.7500, -0.0350,  0.0300],
        [ 0.7500,  0.0350,  0.0000],
        [ 0.7500,  0.0350,  0.0300]])


In [6]:
cube.surface.get_rgba()

ColorTexture(color=(1.0, 1.0, 1.0, 1.0))

In [72]:
cube_pos = (0.7, 0, 0.015)
cube_qpos = np.concatenate((np.array(cube_pos), np.array([1, 0, 0, 0])), axis=0)
cube.set_qpos(cube_qpos)
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])

franka.ready_qpos = qr
franka.set_qpos(franka.ready_qpos)
scene.step()

[Genesis] [18:01:54] [INFO] Running at 0.01 FPS.


In [7]:
# get cylinder properties
cylinder_pos = cylinder.get_pos()
print(f"cylinder_pos: {cylinder_pos}")
cylinder_rot = cylinder.get_quat()
print(f"cylinder_rot: {cylinder_rot}")
cylinder_mass = cylinder.get_mass()
print(f"cylinder_mass: {cylinder_mass}")
# cylinder_vertices = cylinder.get_verts()
# print(f"cylinder_vertices: {cylinder_vertices}")

cylinder_pos: tensor([ 0.7000, -0.2000,  0.0500])
cylinder_rot: tensor([1., 0., 0., 0.])
cylinder_mass: 0.05618601274064494


In [8]:
cylinder.get_AABB()

tensor([[ 0.6700, -0.2300,  0.0000],
        [ 0.7300, -0.1700,  0.1000]])

In [21]:
# get cylinder Rotation matrix
R_initial = gs.quat_to_R(cylinder.get_quat())
R_initial = R_initial.numpy()
# now multiply this R by another given R to get the new rotation matrix
R =np.array([[0, 0, -1], [0, 1, 0], [1, 0, 0]])
R_goal = np.dot(R_initial, R)
quat_goal = gs.R_to_quat(R_goal)
quat_goal

array([-7.05524893e-01, -2.25481084e-10,  7.08685138e-01,  2.71756043e-10])

In [10]:
cylinder.get_quat()

tensor([1., 0., 0., 0.])

In [22]:
cylinder_pos = (0.7, -0.2, 0.05-0.02)
cyl_qpos = np.concatenate((np.array(cylinder_pos), quat_goal), axis=0)
cylinder.set_qpos(cyl_qpos)
scene.step()

[Genesis] [18:58:45] [INFO] Running at 0.01 FPS.


In [77]:
vertices = cylinder.get_verts()
vertices = np.array(vertices)
# # verts.max(axis=-2, keepdim=True)
# verts.max(axis=-2, keepdim=True)

# verts.max(dim = 0).values
# # sort the vertices by x axis
# verts = verts[verts[:, 0].argsort()]
# verts
max_x = np.max(vertices[:, 0])
min_y = np.min(vertices[:, 1])
min_z = np.min(vertices[:, 2])
max_y = np.max(vertices[:, 1])

v1 = np.array([max_x, min_y, min_z])  # max x, min y, min z
v2 = np.array([max_x, max_y, min_z])  # max x, max y, min z
v1

C:\Users\boroj\AppData\Local\Temp\ipykernel_13512\599004262.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  vertices = np.array(vertices)


array([ 0.75049263, -0.23      , -0.00093357], dtype=float32)

In [78]:
v2

array([ 0.75049263, -0.17      , -0.00093357], dtype=float32)

In [23]:
cylinder.get_AABB()

tensor([[ 0.6501, -0.2300, -0.0009],
        [ 0.7505, -0.1700,  0.0598]])

In [13]:
cylinder.get_pos()

tensor([ 0.7003, -0.2000,  0.0294])

In [ ]:
cylinder.set_quat(np.array([1, 0, 0, 0]))

scene.step()

[Genesis] [18:38:02] [INFO] Running at 0.04 FPS.


In [20]:
cylinder.get_ang()

tensor([ 7.0320e-08, -4.4692e-01, -6.5443e-09])

In [32]:
end_effector = franka.get_link('hand')
left_finger = franka.get_link('left_finger')
right_finger = franka.get_link('right_finger')

left_finger.get_pos(), right_finger.get_pos(), end_effector.get_pos()

(tensor([ 0.4585, -0.0406,  0.3952]),
 tensor([0.4582, 0.0394, 0.3948]),
 tensor([ 4.5857e-01, -2.7521e-04,  4.5343e-01]))

In [33]:

# get the end-effector link
end_effector = franka.get_link('hand')
above_box = cube.get_pos().cpu().numpy() + np.array([-cube_size[0]/2 + 0.02, 0, 0.11 + 0.1])
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
# # gripper open pos
# qpos[-2:] = 0.04
# path = franka.plan_path(
#     qpos_goal     = qpos,
#     num_waypoints = 200, # 2s duration
# )

In [ ]:
franka.get_qpos(), qpos

# find straight path in joint space
s = np.linspace(0, 1, 200)
current_pose = franka.get_qpos().cpu().numpy()
goal_pos = qpos.cpu().numpy()
path = current_pose + s[:, None] * (goal_pos - current_pose)


In [ ]:
for qpos in path:
    # set qpos
    franka.control_dofs_position(qpos)
    # step simulation
    scene.step()

In [ ]:
qpos

In [ ]:
# # execute the planned path
# for waypoint in path:
#     franka.control_dofs_position(waypoint)
#     scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    scene.step()


In [ ]:
side_box = above_box + np.array([0.0, 0.0, -0.1])


# reach
qpos_reach = franka.inverse_kinematics(
    link = end_effector,
    pos  = side_box,
    quat = np.array([0, 1, 0, 0]),
)
# path = franka.plan_path(
#     qpos_goal     = qpos_reach,
#     num_waypoints = 100, # 1s duration
# )

# for waypoint in path:
franka.control_dofs_position(qpos_reach)
    # scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

In [ ]:



# grasp
franka.control_dofs_position(qpos_reach[:-2], motors_dof)
franka.control_dofs_force(np.array([-0.5, -0.5]), fingers_dof)

for i in range(100):
    scene.step()


In [ ]:

# lift
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.28]),
    quat=np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
for i in range(200):
    scene.step()

In [ ]:
# open
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.28]),
    quat=np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
for i in range(100):
    scene.step()
# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

In [ ]:
franka.get_dofs_force()

In [ ]:
cube_pos = cube.get_pos().cpu().numpy()
cube_quat = cube.get_quat().cpu().numpy()

cube_pos, cube_quat

In [ ]:
cube_pos = cube.get_pos().cpu().numpy()
cube_quat = cube.get_quat().cpu().numpy()

qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=cube_pos + np.array([0, 0, 0.3]),
    quat=np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
for i in range(100):
    scene.step()
# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

In [ ]:

cam.start_recording()
import numpy as np

for i in range(120):
    scene.step()
    cam.set_pose(
        pos    = (3.0 * np.sin(i / 60), 3.0 * np.cos(i / 60), 2.5),
        lookat = (0, 0, 0.5),
    )
    cam.render()
cam.stop_recording(save_to_filename='video.mp4', fps=60)